In [1]:
# Run this if yfinance is not installed
# !pip install yfinance

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime
import torch
from torch.utils.data import Dataset, DataLoader

In [3]:
def download(ticker = "^GSPC", start_date = None, end_date = None, price = ["Adj Close", "Close", "High", "Low", "Open", "Volume"]):
    df = yf.download(ticker, start = start_date, end = end_date)
    return df[price]

In [4]:
class YahooDataset(Dataset):
    def __init__(self, data):
        self.data = torch.tensor(data)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Config

In [5]:
# ticker = "AAPL, ^GSPC, AMZN"
start_date = "2023-02-01"
end_date = "2024-02-01"
price = 'Close'

batch_size = 64

# Data preparation: generating training and test datasets

In [15]:
df = download(ticker = "^GSPC", start_date = start_date, end_date = end_date, price = price)
date = df.index.strftime("%Y/%m/%d")

split_index = int(len(date)*0.8)
train = df[:split_index]
test = df[split_index:]

train = YahooDataset(train)
test = YahooDataset(test) 

[*********************100%%**********************]  1 of 1 completed

(200,) torch.Size([200])


In [21]:
train_loader = DataLoader(train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test, batch_size = batch_size, shuffle = False)
for i, batch in enumerate(dataloader):
    print(batch)

tensor([4119.2100, 4179.7598, 4136.4800, 4111.0801, 4164.0000, 4117.8599,
        4081.5000, 4090.4600, 4137.2900, 4136.1299, 4147.6001, 4090.4099,
        4079.0901, 3997.3401, 3991.0500, 4012.3201, 3970.0400, 3982.2400,
        3970.1499, 3951.3899, 3981.3501, 4045.6399, 4048.4199, 3986.3701,
        3992.0100, 3918.3201, 3861.5901, 3855.7600, 3919.2900, 3891.9299,
        3960.2800, 3916.6399, 3951.5701, 4002.8701, 3936.9700, 3948.7200,
        3970.9900, 3977.5300, 3971.2700, 4027.8101, 4050.8301, 4109.3101,
        4124.5098, 4100.6001, 4090.3799, 4105.0200, 4109.1099, 4108.9399,
        4091.9500, 4146.2202, 4137.6401, 4151.3198, 4154.8701, 4154.5200,
        4129.7900, 4133.5200, 4137.0400, 4071.6299, 4055.9900, 4135.3501,
        4169.4800, 4167.8701, 4119.5801, 4090.7500], dtype=torch.float64)
tensor([4061.2200, 4136.2500, 4138.1201, 4119.1699, 4137.6401, 4130.6201,
        4124.0801, 4136.2798, 4109.8999, 4158.7700, 4198.0498, 4191.9800,
        4192.6299, 4145.5801, 4115.240